<h1>ex8-1Titanic</h1>

In [1]:
import torch
import numpy as np
import pandas as pd
from torch.autograd import Variable

In [2]:
#skeleton code for dataset class
class Titanic():
    def __init__(self, dtype):
        self.type=dtype
        if dtype=="train":
            train_data=pd.read_csv("D:/KHU_Desktop/4-2/Machine Learning/data/titanic/train.csv")
        elif dtype=="test":
            train_data=pd.read_csv("D:/KHU_Desktop/4-2/Machine Learning/data/titanic/test.csv")
        train_data['FamilySize'] = train_data['SibSp'] + train_data['Parch'] + 1
        train_data = train_data.drop(columns=['Ticket', 'PassengerId', 'Cabin'])
        train_data['Sex'] = train_data['Sex'].map({'male':0, 'female':1})
        train_data['Embarked'] = train_data['Embarked'].map({'C':0, 'Q':1, 'S':2})
        train_data['Title'] = train_data.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
        train_data = train_data.drop(columns='Name')
        train_data['Title'] = train_data['Title'].replace(['Dr', 'Rev', 'Col', 'Major', 'Countess', 'Sir', 'Jonkheer', 'Lady', 'Capt', 'Don'], 'Others')
        train_data['Title'] = train_data['Title'].replace('Ms', 'Miss')
        train_data['Title'] = train_data['Title'].replace('Mme', 'Mrs')
        train_data['Title'] = train_data['Title'].replace('Mlle', 'Miss')
        train_data['Title'] = train_data['Title'].map({'Master':0, 'Miss':1, 'Mr':2, 'Mrs':3, 'Others':4})
        train_data['Embarked'] = train_data['Embarked'].fillna(2)
        NaN_indexes = train_data['Age'][train_data['Age'].isnull()].index
        for i in NaN_indexes:
            pred_age = train_data['Age'][((train_data.SibSp == train_data.iloc[i]["SibSp"]) & (train_data.Parch == train_data.iloc[i]["Parch"]) & (train_data.Pclass == train_data.iloc[i]["Pclass"]))].median()
            if not np.isnan(pred_age):
                train_data['Age'].iloc[i] = pred_age
            else:
                train_data['Age'].iloc[i] = train_data['Age'].median()

        #print(train_data)
        if dtype=="train":
            self.y_data=Variable(torch.tensor(train_data['Survived'].values, dtype=torch.float32))
            train_data.pop("Survived")
            self.x_data=Variable(torch.tensor(train_data.values, dtype=torch.float32))
        else:
            self.x_data=Variable(torch.tensor(train_data.values))
        self.len=len(train_data)
        
    def __getitem__(self, index):
        if self.type=="test":
            return self.x_data[index]
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

In [3]:
train=Titanic("train")
train_loader=torch.utils.data.DataLoader(dataset=train, batch_size=32, shuffle=True)

C:\conda\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
class TitanicNet(torch.nn.Module):
    def __init__(self):
        super(TitanicNet, self).__init__()
        self.l1=torch.nn.Linear(9, 16)
        self.l2=torch.nn.Linear(16, 32)
        self.l3=torch.nn.Linear(32, 16)
        self.l4=torch.nn.Linear(16, 1)
        self.sigmoid=torch.nn.Sigmoid()
    def forward(self, x):
        out1=self.l1(x)
        out2=self.l2(out1)
        out3=self.l3(out2)
        out4=self.l4(out3)
        return self.sigmoid(out4)

In [5]:
titanic=TitanicNet()
criterion=torch.nn.BCELoss(reduction='mean')
optimizer=torch.optim.Adam(titanic.parameters())

In [6]:
for epoch in range(10):
    for idx, data in enumerate(train_loader):
        inputs, labels=data
        y_pred=titanic(inputs)
        loss=criterion(y_pred, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"EPOCH: {epoch+1} | Loss: {loss.item()}")

C:\conda\lib\site-packages\torch\nn\modules\loss.py:498: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
C:\conda\lib\site-packages\torch\nn\modules\loss.py:498: UserWarning: Using a target size (torch.Size([27])) that is different to the input size (torch.Size([27, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


EPOCH: 1 | Loss: 0.9315544366836548
EPOCH: 2 | Loss: 0.5765495896339417
EPOCH: 3 | Loss: 0.5677251815795898
EPOCH: 4 | Loss: 0.5835548043251038
EPOCH: 5 | Loss: 0.5604218244552612
EPOCH: 6 | Loss: 0.7461932301521301
EPOCH: 7 | Loss: 0.667543888092041
EPOCH: 8 | Loss: 0.6090598106384277
EPOCH: 9 | Loss: 0.5146723985671997
EPOCH: 10 | Loss: 0.613013744354248


In [7]:
test_loader=torch.utils.data.DataLoader(dataset=train, batch_size=1, shuffle=True)
correct=0
cnt=0

for idx, data in enumerate(test_loader):
    inputs, labels=data
    y_pred=torch.round(titanic(inputs))
    cnt+=1
    if(y_pred.item() == labels.item()):
        correct+=1

print(f"accuracy: {correct}/{cnt} | {correct/cnt*100}%")

accuracy: 619/891 | 69.47250280583613%


<h1>ex9-1 Difference between Cross enctropy loss and NLLLoss</h1>

cross entropy loss  는 Log softmax와 NLLLoss함수를 동시에 criterion으로 사용하는 것이다. 

<h1>ex9 otto dataset</h1>

In [8]:
from torch import nn
import torch.nn.functional as F

In [9]:
def get_class(df):
    return int(df['target'][-1:])
data=pd.read_csv("D:/KHU_Desktop/4-2/Machine Learning/data/train.csv")
y=data.apply(get_class, axis=1).values

In [19]:
class otto:
    def __init__(self):
        def get_class(df):
            return int(df['target'][-1:])
        data=pd.read_csv("D:/KHU_Desktop/4-2/Machine Learning/data/train.csv")
        
        data.pop('id')
        y=data.apply(get_class, axis=1).values
        ylist=[]
        for i in y:
            if i==1:
                ylist.append([1,0,0,0,0,0,0,0,0])
            elif i==2:
                ylist.append([0,1,0,0,0,0,0,0,0])
            elif i==3:
                ylist.append([0,0,1,0,0,0,0,0,0])
            elif i==4:
                ylist.append([0,0,0,1,0,0,0,0,0])
            elif i==5:
                ylist.append([0,0,0,0,1,0,0,0,0])
            elif i==6:
                ylist.append([0,0,0,0,0,1,0,0,0])
            elif i==7:
                ylist.append([0,0,0,0,0,0,1,0,0])
            elif i==8:
                ylist.append([0,0,0,0,0,0,0,1,0])
            elif i==9:
                ylist.append([0,0,0,0,0,0,0,0,1])
        
        
        self.y_data=torch.autograd.Variable(torch.tensor(ylist))
        data.pop('target')
        self.x_data=torch.autograd.Variable(torch.tensor(data.values, dtype=torch.float32))
        self.len=len(self.x_data)
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

In [20]:
data=otto()

In [21]:
data_loader=torch.utils.data.DataLoader(dataset=data, batch_size=32, shuffle=True)

In [22]:
class ottoNet(nn.Module):
    def __init__(self):
        super(ottoNet, self).__init__()
        self.l1=nn.Linear(93, 128)
        self.l2=nn.Linear(128, 64)
        self.l3=nn.Linear(64, 32)
        self.l4=nn.Linear(32, 1)
        self.soft=nn.Softmax()
    
    def forward(self, x):
        out1=F.relu(self.l1(x))
        out2=F.relu(self.l2(out1))
        out3=F.relu(self.l3(out2))
        out4=F.relu(self.l4(out3))
        return self.soft(out4)

In [25]:
otto=ottoNet()
criterion=nn.MultiLabelSoftMarginLoss()
optimizer=torch.optim.SGD(otto.parameters(), lr=0.01)

In [27]:
for epoch in range(15):
    for idx, data in enumerate(data_loader):
        sample, label=data
        #print(sample.size(), label)
        y_pred=otto(sample)
        loss=criterion(y_pred, label)
        #change label to one hot vector
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f"EPOCH: {epoch+1} | Loss: {loss.item()}")

C:\conda\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


EPOCH: 1 | Loss: 10.819352149963379
EPOCH: 2 | Loss: 10.819352149963379
EPOCH: 3 | Loss: 10.819352149963379
EPOCH: 4 | Loss: 10.819352149963379
EPOCH: 5 | Loss: 10.819352149963379


KeyboardInterrupt: 